In [2]:
import requests
from bs4 import BeautifulSoup
import datetime
import csv
import pandas as pd
import re

In [27]:
# Constants
News_URL = "https://www.prnewswire.com/news-releases/news-releases-list/?"
#News_URL = "https://www.prnewswire.com/news-releases/news-releases-list/?month=11&day=24&year=2021&hour=00&page=1&pagesize=100"

Total_Days = 7 #Number of days to fetch
StockIndustry_Search = 'NYSE'
StockSymbol_Required = 2

In [4]:
# Global Variables
arr_Date = []
arr_newsTitle = []
arr_newsPara = []

In [5]:
# Date for extracting webpage
currentDate = datetime.datetime.now()
tempDate = currentDate

In [6]:
def prepare_URL(dayToFetch) -> str:
   urlByDay = News_URL + 'month=' + str(currentDate.month) + '&day=' + str(dayToFetch) + '&year=' + str(currentDate.year) + '&hour=00&page=1&pagesize=100'
   return urlByDay

In [7]:
def parse_News(forDay):
  url = prepare_URL(forDay)
  page = requests.get(url)
  soup = BeautifulSoup(page.content, "html.parser")

  # to extract main layout content
  main = soup.find_all('main',class_='headline-listing')

  for elements in main:
    lstContainer = elements.find_all('section',class_='container')
    for mainHead in lstContainer:
      row = mainHead.find_all('div',class_='row')
      for mainCol in row:
        rowList = mainCol.find_all('div',class_='col-md-8 col-sm-8 card-list card-list-hr')
        for newsCard in rowList:
          newsLst = newsCard.find_all('a',class_='newsreleaseconsolidatelink')
          for content in newsLst:
            small = content.h3.small.text
            content.h3.small.decompose()
            h3 = content.h3.text.replace("\n", "")
            p =  content.p.text
            arr_Date.append(small)
            arr_newsTitle.append(h3)
            arr_newsPara.append(p)

In [8]:
#Fetch & Parse data of last week
i = 0
while i < Total_Days:
  parse_News(currentDate.day - i)
  i = i + 1

In [9]:
# Write data to csv
dictNews = {'Date': arr_Date, 'Title': arr_newsTitle, 'Content': arr_newsPara}
df = pd.DataFrame(dictNews)
df.to_csv('newsData.csv')

df.head()

,Date,Title,Content
0,23:19 ET,SHAREHOLDER ALERT: Pomerantz Law Firm Reminds ...,Pomerantz LLP announces that a class action la...
1,23:19 ET,SHAREHOLDER ALERT: Pomerantz Law Firm Investig...,Pomerantz LLP is investigating claims on behal...
2,23:17 ET,SHAREHOLDER ALERT: Pomerantz Law Firm Reminds ...,Pomerantz LLP announces that a class action la...
3,23:16 ET,SHAREHOLDER ALERT: Pomerantz Law Firm Investig...,Pomerantz LLP is investigating claims on behal...
4,23:15 ET,SHAREHOLDER ALERT: Pomerantz Law Firm Investig...,Pomerantz LLP is investigating claims on behal...


In [ ]:
#Search stock symbols of given industry

searchString = StockIndustry_Search + ': ([a-zA-Z]+)'
df_stockSymbols = df['Content'].str.extract(searchString, expand=True)

# Dropping the duplicates 
df_stockSymbols = df_stockSymbols.drop_duplicates()

# Dropping the missing values.
df_stockSymbols = df_stockSymbols.dropna() 

#Yahoo Finance API
